# Création du model de machine learning


#### Étude des variations des données historiques dans le but de prédire si le prix va augmenter ou diminuer


- utilisation des méthodes de classification de l'apprentissage supervisé

- calcule de la variation du prix pour chaque période de temps (chaque bougie dans notre cas)

- utilisation du calcul comme valeur cible(étiquette) dans les données d'apprentissage supervisé


#### Définition des valeurs explicatives (caractéristiques)


- Prix d'ouverture (open)

- Prix le plus haut (high)

- Prix le plus bas (low)

- Volume de transactions (volume)


#### Définition de la valeur cible (étiquette)


La valeur cible sera une variable binaire indiquant si le prix a augmenté ou diminué dans la période de temps suivante entre chaque bougie :

- 1 si le prix a augmenté.
- 0 si le prix a diminué ou est resté inchangé.


#### Prépocessing


In [1]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings("ignore")

In [2]:
# MongoClient pour interagir avec MongoDB
from pymongo import MongoClient

# authentification à MongoDB
# définition des informations d'identification nécessaires
# pour s'authentifier auprès de MongoDB en local
mongo_user = "admin"
mongo_password = "pass"
mongo_host = "localhost"
mongo_port = 27017


# connexion à la base de données MongoDB
client = MongoClient(
    f"mongodb://{mongo_user}:{mongo_password}@{mongo_host}:{mongo_port}/"
)

# sélection de la base de données
db = client["extract_data_binance"]

# sélection de la collection
collection = db["historical_data"]

In [3]:
# récupération des données de la collection
# et stockage dans un DataFrame
df = pd.DataFrame(list(collection.find()))

# affichage de la taille du dataframe
df.shape

ServerSelectionTimeoutError: localhost:27017: [Errno 111] Connection refused (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30s, Topology Description: <TopologyDescription id: 65f9ad43794156ebafd3d470, topology_type: Unknown, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [Errno 111] Connection refused (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>]>

In [ ]:
# affichage des 5 premières lignes
df.head()

,_id,symbol,timestamp,open,high,low,close,volume
0,65f589d4163e278288439f2e,BTCUSDT,2024-03-16 11:43:41,68350.56,68350.57,68350.56,68350.57,0.11233
1,65f589d4163e278288439f2f,BTCUSDT,2024-03-16 11:43:42,68350.57,68350.57,68350.56,68350.56,0.65264
2,65f589d4163e278288439f30,BTCUSDT,2024-03-16 11:43:43,68350.56,68350.57,68350.56,68350.57,0.17315
3,65f589d4163e278288439f31,BTCUSDT,2024-03-16 11:43:44,68350.56,68350.57,68344.28,68344.28,0.47940
4,65f589d4163e278288439f32,BTCUSDT,2024-03-16 11:43:45,68344.28,68344.29,68344.28,68344.28,0.05781


In [ ]:
# vérification des doublons
doublons = df.duplicated().sum()
print(f"le dataframe contient {doublons} doublons")

le dataframe contient 0 doublons


In [ ]:
# vérification des valeurs manquantes
display(df.isna().sum())

_id          0
symbol       0
timestamp    0
open         0
high         0
low          0
close        0
volume       0
dtype: int64

In [ ]:
# vérification des types de données
df.dtypes

_id                  object
symbol               object
timestamp    datetime64[ns]
open                float64
high                float64
low                 float64
close               float64
volume              float64
dtype: object

In [ ]:
# description des statistiques quantitatives
df.describe()

,timestamp,open,high,low,close,volume
count,2000,2000.000000,2000.00000,2000.000000,2000.00000,2000.000000
mean,2024-03-17 09:42:26.500000,68293.969350,68295.13482,68292.715945,68293.88290,0.491492
min,2024-03-16 11:43:41,68127.630000,68127.63000,68123.790000,68127.62000,0.000000
25%,2024-03-16 11:52:00.750000128,68239.990000,68240.00000,68239.700000,68239.99000,0.040895
50%,2024-03-17 09:42:26.500000,68288.330000,68290.00000,68287.930000,68288.23000,0.179925
75%,2024-03-18 07:32:52.249999872,68341.090000,68341.10000,68340.050000,68341.09000,0.553430
max,2024-03-18 07:41:12,68457.660000,68458.25000,68457.660000,68457.67000,16.362230
std,NaN,72.651524,72.61287,72.696158,72.65521,1.001394


In [ ]:
# calcul du nombre de modalité(valeurs différentes)pour chaque variable explicative
modalite_par_variable = df.nunique()

# affichage des modalités
print(modalite_par_variable)

_id          2000
symbol          1
timestamp    2000
open          770
high          736
low           702
close         807
volume       1918
dtype: int64


#### Calcul des variations du prix pour créer la variable cible(étiquette) dans les données d'apprentissage supervisé

- utilisation de la fonction pct_change()
- c'est une méthode de la librairie pandas
- qui permet de calculer le pourcentage de changement
- entre les valeurs successives d'une série ou d'un dataframe.
- cette fonction est couramment utilisée pour
- calculer les rendements ou les variations pour les séries temporelles financières

Pour une série temporelle ou une colonne d'un DataFrame, la fonction pct_change() calcule le pourcentage de changement entre chaque élément et son précédent.

Elle renvoie une nouvelle série ou un nouveau DataFrame où chaque élément représente le pourcentage de changement par rapport à son prédécesseur.

NaN (Not a Number) est renvoyé pour la première ligne car il n'y a pas de valeur précédente à partir de laquelle calculer le pourcentage de changement.


In [ ]:
# calcul de la variation journalière
df["taux_variation"] = df["close"].pct_change()

# affichage des 5 premières lignes
df.head()

,_id,symbol,timestamp,open,high,low,close,volume,taux_variation
0,65f589d4163e278288439f2e,BTCUSDT,2024-03-16 11:43:41,68350.56,68350.57,68350.56,68350.57,0.11233,NaN
1,65f589d4163e278288439f2f,BTCUSDT,2024-03-16 11:43:42,68350.57,68350.57,68350.56,68350.56,0.65264,-1.463046e-07
2,65f589d4163e278288439f30,BTCUSDT,2024-03-16 11:43:43,68350.56,68350.57,68350.56,68350.57,0.17315,1.463046e-07
3,65f589d4163e278288439f31,BTCUSDT,2024-03-16 11:43:44,68350.56,68350.57,68344.28,68344.28,0.47940,-9.202557e-05
4,65f589d4163e278288439f32,BTCUSDT,2024-03-16 11:43:45,68344.28,68344.29,68344.28,68344.28,0.05781,0.000000e+00


In [ ]:
# création d'une nouvelle colonne pour la valeur cible binaire
# ajoute 1 si le prix a augmenté.
# ajoute 0 si le prix a diminué ou est resté inchangé.

df["target"] = (df["taux_variation"] > 0).astype(int)

# affichage des 5 premières lignes pour vérification
df.head()

,_id,symbol,timestamp,open,high,low,close,volume,taux_variation,target
0,65f589d4163e278288439f2e,BTCUSDT,2024-03-16 11:43:41,68350.56,68350.57,68350.56,68350.57,0.11233,NaN,0
1,65f589d4163e278288439f2f,BTCUSDT,2024-03-16 11:43:42,68350.57,68350.57,68350.56,68350.56,0.65264,-1.463046e-07,0
2,65f589d4163e278288439f30,BTCUSDT,2024-03-16 11:43:43,68350.56,68350.57,68350.56,68350.57,0.17315,1.463046e-07,1
3,65f589d4163e278288439f31,BTCUSDT,2024-03-16 11:43:44,68350.56,68350.57,68344.28,68344.28,0.47940,-9.202557e-05,0
4,65f589d4163e278288439f32,BTCUSDT,2024-03-16 11:43:45,68344.28,68344.29,68344.28,68344.28,0.05781,0.000000e+00,0


In [ ]:
# vérification des valeurs manquantes
display(df.isna().sum())

_id               0
symbol            0
timestamp         0
open              0
high              0
low               0
close             0
volume            0
taux_variation    1
target            0
dtype: int64

In [ ]:
# suppression des colonnes non nécessaire au machine learning
# "timestamp" car nous avons effectué le calcul de la variation journalière
# "_id" car c'est un identifiant unique
# "symbol" car nous avons un seul symbole
df = df.drop(["timestamp", "_id", "symbol"], axis=1)

# affichage pour vérification
df.head()

,open,high,low,close,volume,taux_variation,target
0,68350.56,68350.57,68350.56,68350.57,0.11233,NaN,0
1,68350.57,68350.57,68350.56,68350.56,0.65264,-1.463046e-07,0
2,68350.56,68350.57,68350.56,68350.57,0.17315,1.463046e-07,1
3,68350.56,68350.57,68344.28,68344.28,0.47940,-9.202557e-05,0
4,68344.28,68344.29,68344.28,68344.28,0.05781,0.000000e+00,0


In [ ]:
#  séparation de la variable cible des variables explicatives
feats = df.drop("target", axis=1)
target = df["target"]

# affichage pour vérification
# feats.head()
# target.head()

In [ ]:
# importation de la fonction train_test_split
# pour séparer le jeu de données en données d'entraînement et données de test
from sklearn.model_selection import train_test_split

# séparation du jeu d'entrainement et du jeu de test
X_train, X_test, y_train, y_test = train_test_split(
    feats, target, test_size=0.20, random_state=42
)

# vérification des tailles du jeu de données
print("train Set:", X_train.shape)
print("test Set:", X_test.shape)

train Set: (1600, 6)
test Set: (400, 6)


In [ ]:
# vérification du type des variables
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   open            2000 non-null   float64
 1   high            2000 non-null   float64
 2   low             2000 non-null   float64
 3   close           2000 non-null   float64
 4   volume          2000 non-null   float64
 5   taux_variation  1999 non-null   float64
 6   target          2000 non-null   int64  
dtypes: float64(6), int64(1)
memory usage: 109.5 KB


In [ ]:
df.head()

,open,high,low,close,volume,taux_variation,target
0,68350.56,68350.57,68350.56,68350.57,0.11233,NaN,0
1,68350.57,68350.57,68350.56,68350.56,0.65264,-1.463046e-07,0
2,68350.56,68350.57,68350.56,68350.57,0.17315,1.463046e-07,1
3,68350.56,68350.57,68344.28,68344.28,0.47940,-9.202557e-05,0
4,68344.28,68344.29,68344.28,68344.28,0.05781,0.000000e+00,0


In [ ]:
# il n'y a pas de variables catégorielles donc pas besoin de faire de séparation entre les variables numériques et catégorielles

In [ ]:
# affichage vérifications des valeurs manquantes
# dans les jeux de données d'entraînement et de test
# des variables explicatives
print("valeurs manquantes dans X_train:")
print(X_train.isna().sum())

print("valeurs manquantes dans X_test:")
print(X_test.isna().sum())

# affichage vérifications des valeurs manquantes
# dans les jeux de données d'entraînement et de test
# de la cible
print("valeurs manquantes dans y_train:")
print(y_train.isna().sum())

print("valeurs manquantes dans y_test:")
print(y_test.isna().sum())

valeurs manquantes dans X_train:
open              0
high              0
low               0
close             0
volume            0
taux_variation    1
dtype: int64
valeurs manquantes dans X_test:
open              0
high              0
low               0
close             0
volume            0
taux_variation    0
dtype: int64
valeurs manquantes dans y_train:
0
valeurs manquantes dans y_test:
0


In [ ]:
# remplissage des valeurs manquantes dans la variable cible
from sklearn.impute import SimpleImputer

# création de l'imputeur pour les variables manquantes numériques avec la stratégie "median"
num_imputer = SimpleImputer(missing_values=np.nan, strategy="median")

# application de l'imputeur sur X_train et X_test
X_train_imputed = pd.DataFrame(
    num_imputer.fit_transform(X_train), columns=X_train.columns
)

X_test_imputed = pd.DataFrame(num_imputer.transform(X_test), columns=X_test.columns)

# affichage vérifications des valeurs manquantes numériques
print("valeurs manquantes numériques dans X_train:")
print(X_train_imputed.isna().sum())

print("valeurs manquantes numériques dans X_test:")
print(X_test_imputed.isna().sum())

valeurs manquantes numériques dans X_train:
open              0
high              0
low               0
close             0
volume            0
taux_variation    0
dtype: int64
valeurs manquantes numériques dans X_test:
open              0
high              0
low               0
close             0
volume            0
taux_variation    0
dtype: int64


In [ ]:
# il n'y a pas d'encodage à faire car il n'y a pas de variables catégorielles

In [ ]:
# assignation des variables nettoyées
X_train_clean = X_train_imputed
X_test_clean = X_test_imputed

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

# initialisation du modèle de régression logistique
model_rl = LogisticRegression()

# entrainement du modèle régression logistique
model_rl.fit(X_train_clean, y_train)

# initialisation du modèle arbre de décision
model_ad = DecisionTreeClassifier()

# entrainement du modèle arbre de décision
model_ad.fit(X_train_clean, y_train)

DecisionTreeClassifier()

In [ ]:
# évaluation de la performande des modèles
print(
    "Accuracy score régression logistique : ",
    model_rl.score(X_test_clean, y_test),
)
print("Accuracy score arbre de décision : ", model_ad.score(X_test_clean, y_test))
print("le modèle régression logistique est le plus performant")

Accuracy score régression logistique :  0.8425
Accuracy score arbre de décision :  1.0
le modèle régression logistique est le plus performant


Prédictions :

Les prédictions pour les données de test sont stockées dans la variable y_pred_test.

Les données de test sont stockées dans la variable y_test.


In [ ]:
from sklearn.metrics import classification_report

# prédiction sur le jeu de test "historical_data.csv" avec le modèle régression logistique car c'est le plus performant
y_pred_test = model_rl.predict(X_test_clean)

In [ ]:
# affichage de la matrice de confusion sur "historical_data.csv"
display(
    pd.crosstab(
        y_test, y_pred_test, colnames=["Classes réelles"], rownames=["Classes prédites"]
    )
)

Classes réelles,0,1
Classes prédites,,
0,274,0
1,63,63


La conclusion des prédictions est la suivante :

Les prédictions ont été effectuées avec succès en utilisant le modèle de classification LogisticRegression car c'est le plus performant.

Les prédictions ont été comparées aux valeurs réelles des données de test.

Les résultats des prédictions sont stockés dans la variable y_pred_test.

Nous avons :
139 Vrais négatifs.
0 Faux négatifs.
20 Vrais positifs.
41 Faux positifs.

Ces prédictions peuvent être utilisées pour évaluer la performance des modèles de classification et prendre des décisions d'achats ou de ventes basées sur les résultats obtenus.


In [ ]:
# affichage du rapport de classification sur "historical_data.csv"
print(classification_report(y_test, y_pred_test))

              precision    recall  f1-score   support

           0       0.81      1.00      0.90       274
           1       1.00      0.50      0.67       126

    accuracy                           0.84       400
   macro avg       0.91      0.75      0.78       400
weighted avg       0.87      0.84      0.82       400



In [ ]:
# définition d'une pipeline pour le prétraitement des données ou pas ?
from sklearn.pipeline import Pipeline